In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pendulum

%matplotlib inline

In [2]:
import data
import color
import categories
import regression
import echarts
from cytoolz import *
from cytoolz.itertoolz import accumulate
from operator import add

In [3]:
sales =  [
    data.read_store_sales('F01451', begin='2017-07-01', end='2018-12-31'),
    data.read_store_sales('F01450', begin='2017-07-01', end='2018-12-31')
]

In [4]:
sales[0]['margin'] = sales[0]['profit'] / sales[0]['sales'] * 100
sales[1]['margin'] = sales[1]['profit'] / sales[1]['sales'] * 100
sales[0] = sales[0].replace([np.inf, -np.inf], np.nan).dropna()
sales[1] = sales[1].replace([np.inf, -np.inf], np.nan).dropna()

In [23]:
person = 7
s1 = sales[0].resample('M', on='date').agg({'sales': 'sum', 'profit': 'sum'})
s1['loss'] = sales[0][(sales[0]['margin'] < -100) & (sales[0]['sales'] > 0)].resample('M', on='date')['profit'].sum()
s1['total_bonus'] = s1['profit'] * 0.1
s1['per_person'] = s1['total_bonus'] / (person + 1)
s1['director'] = np.round((1.8 * s1['per_person']), 2)
s1['director'] = s1['director'].apply(lambda x: min(x, 650))
s1['meat_director'] = np.round((1.3 * s1['per_person']), 2)
s1['meat_director'] = s1['meat_director'].apply(lambda x: min(x, 450))
s1['bonus'] = s1['total_bonus'] - s1['director'] - s1['meat_director']
s1['per_person'] = s1['bonus'] / (person - 2)

In [24]:
s1.tail()

,sales,profit,loss,total_bonus,per_person,director,meat_director,bonus
date,,,,,,,,
2018-02-28,168685.48,31417.43,-1238.46,3141.743,408.3486,650.0,450.0,2041.743
2018-03-31,211493.81,39486.30,-1759.86,3948.630,569.7260,650.0,450.0,2848.630
2018-04-30,184776.30,35078.26,-1920.18,3507.826,481.5652,650.0,450.0,2407.826
2018-05-31,208474.43,40804.71,-1791.30,4080.471,596.0942,650.0,450.0,2980.471
2018-06-30,214018.32,39557.03,-2293.43,3955.703,571.1406,650.0,450.0,2855.703


475.8333333333333

In [27]:
person = 5
s3 = sales[1].resample('M', on='date').agg({'sales': 'sum', 'profit': 'sum'})
s3['loss'] = sales[1][(sales[1]['margin'] < -100) & (sales[1]['sales'] > 0)].resample('M', on='date')['profit'].sum()
s3['total_bonus'] = s3['profit'] * 0.1
s3['per_person'] = s3['total_bonus'] / (person + 1)
s3['director'] = np.round((1.8 * s3['per_person']), 2)
s3['director'] = s3['director'].apply(lambda x: min(x, 650))
s3['meat_director'] = np.round((1.3 * s3['per_person']), 2)
s3['meat_director'] = s3['meat_director'].apply(lambda x: min(x, 450))
s3['bonus'] = s3['total_bonus'] - s3['director'] - s3['meat_director']
s3['per_person'] = s3['bonus'] / (person - 2)

s3.tail()

,sales,profit,loss,total_bonus,per_person,director,meat_director,bonus
date,,,,,,,,
2018-02-28,94169.07,16243.51,-472.57,1624.351,261.700333,487.31,351.94,785.101
2018-03-31,119851.89,20513.09,-481.19,2051.309,330.489667,615.39,444.45,991.469
2018-04-30,101234.66,17484.51,-1121.17,1748.451,281.693667,524.54,378.83,845.081
2018-05-31,116143.52,19709.88,-1594.77,1970.988,317.546000,591.30,427.05,952.638
2018-06-30,129624.68,22173.55,-1542.19,2217.355,372.451667,650.00,450.00,1117.355
